# AI Portfolio Generator - Database Design & Authentication System

This notebook demonstrates the complete database architecture for the AI Portfolio Generator application, including:

- **User Authentication** with secure password hashing
- **User Profile Management** with comprehensive signup data storage
- **Portfolio Generation** data management with JSON structure
- **Secure Login System** with credential verification

## Key Features:
- ✅ **Hashed Password Storage** - Secure bcrypt password hashing
- ✅ **Complete User Profiles** - All signup form data in one table
- ✅ **Portfolio JSON Storage** - Structured portfolio generation data
- ✅ **Professional Database Design** - Proper relationships and constraints

## 1. Database Connection Setup

First, let's establish our database connection and import the required libraries.

In [1]:
# Import required libraries
import os
import json
import uuid
from datetime import datetime, timezone
import bcrypt
import hashlib

# Database connection (for Supabase/PostgreSQL)
try:
    import psycopg2
    from psycopg2.extras import RealDictCursor
    print("✅ psycopg2 imported successfully")
except ImportError:
    print("❌ psycopg2 not installed. Run: pip install psycopg2-binary")

# For demonstration purposes - database connection parameters
DB_CONFIG = {
    "host": "your-supabase-host.supabase.co",
    "port": 5432,
    "database": "postgres",
    "user": "postgres",
    "password": "your-password"
}

print("🔧 Database configuration ready")
print("📊 Libraries imported for secure authentication and data management")

❌ psycopg2 not installed. Run: pip install psycopg2-binary
🔧 Database configuration ready
📊 Libraries imported for secure authentication and data management


## 2. Password Hashing and Security

Implement secure password hashing using bcrypt to ensure user passwords are never stored in plain text.

In [2]:
class PasswordManager:
    """Secure password hashing and verification using bcrypt"""
    
    @staticmethod
    def hash_password(password: str) -> str:
        """
        Hash a password using bcrypt with salt
        
        Args:
            password (str): Plain text password
            
        Returns:
            str: Hashed password suitable for database storage
        """
        # Generate salt and hash password
        salt = bcrypt.gensalt()
        hashed = bcrypt.hashpw(password.encode('utf-8'), salt)
        return hashed.decode('utf-8')
    
    @staticmethod
    def verify_password(password: str, hashed_password: str) -> bool:
        """
        Verify a password against its hash
        
        Args:
            password (str): Plain text password to verify
            hashed_password (str): Stored hashed password
            
        Returns:
            bool: True if password matches, False otherwise
        """
        return bcrypt.checkpw(password.encode('utf-8'), hashed_password.encode('utf-8'))

# Demonstration of password hashing
demo_password = "MySecurePassword123!"
hashed_demo = PasswordManager.hash_password(demo_password)

print("🔐 Password Security Demo:")
print(f"Original Password: {demo_password}")
print(f"Hashed Password: {hashed_demo}")
print(f"Hash Length: {len(hashed_demo)} characters")
print(f"Verification Test: {PasswordManager.verify_password(demo_password, hashed_demo)}")
print(f"Wrong Password Test: {PasswordManager.verify_password('wrongpassword', hashed_demo)}")

🔐 Password Security Demo:
Original Password: MySecurePassword123!
Hashed Password: $2b$12$BIgcdRtJwNmwWIzL5nBpYerVaA4VrhobT/x29/Z8K7G09sK6JHOwS
Hash Length: 60 characters
Verification Test: True
Wrong Password Test: False


## 3. User Registration Workflow

Create functions to handle complete user registration with all signup form data.

In [3]:
class UserRegistration:
    """Handle user registration with comprehensive profile data"""
    
    @staticmethod
    def create_user_profile(user_data: dict) -> dict:
        """
        Create a new user profile with all signup form data
        
        Args:
            user_data (dict): Complete signup form data
            
        Returns:
            dict: Result of registration process
        """
        try:
            # Generate unique user ID
            user_id = str(uuid.uuid4())
            
            # Hash the password
            hashed_password = PasswordManager.hash_password(user_data['password'])
            
            # Prepare user profile data
            profile_data = {
                'id': user_id,
                'email': user_data['email'],
                'password_hash': hashed_password,
                'full_name': user_data['full_name'],
                'mobile_number': user_data.get('mobile_number'),
                'student_id': user_data.get('student_id'),
                'course': user_data.get('course'),
                'year_of_graduation': user_data.get('year_of_graduation'),
                'college_name': user_data.get('college_name'),
                'linkedin_profile': user_data.get('linkedin_profile'),
                'github_profile': user_data.get('github_profile'),
                'profile_completed': True,
                'account_status': 'active',
                'created_at': datetime.now(timezone.utc),
                'updated_at': datetime.now(timezone.utc)
            }
            
            return {
                'success': True,
                'user_id': user_id,
                'profile_data': profile_data,
                'message': 'User profile created successfully'
            }
            
        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'message': 'Failed to create user profile'
            }

# Demo user registration data (matching your signup form)
demo_signup_data = {
    'email': 'hamdeen.a2004@gmail.com',
    'password': 'SecurePassword123!',
    'full_name': 'MOHAMED HAMDEEN',
    'mobile_number': '9043661031',
    'student_id': 'AI2022001',
    'course': 'Artificial Intelligence and Data Science',
    'year_of_graduation': 2026,
    'college_name': 'Aalim Mohammed Salegh College of Engineering',
    'linkedin_profile': 'https://linkedin.com/in/mohamed-hamdeen',
    'github_profile': 'https://github.com/mohamedhamdeen'
}

# Create user profile
registration_result = UserRegistration.create_user_profile(demo_signup_data)

print("👤 User Registration Demo:")
print(f"Success: {registration_result['success']}")
print(f"User ID: {registration_result.get('user_id', 'N/A')}")
print(f"Message: {registration_result['message']}")

if registration_result['success']:
    profile = registration_result['profile_data']
    print(f"\n📋 Profile Data:")
    print(f"  Email: {profile['email']}")
    print(f"  Name: {profile['full_name']}")
    print(f"  Course: {profile['course']}")
    print(f"  College: {profile['college_name']}")
    print(f"  Password Hash: {profile['password_hash'][:20]}...") # Show only first 20 chars

👤 User Registration Demo:
Success: True
User ID: 4091885c-a53a-4bb8-8a96-ab3a00049903
Message: User profile created successfully

📋 Profile Data:
  Email: hamdeen.a2004@gmail.com
  Name: MOHAMED HAMDEEN
  Course: Artificial Intelligence and Data Science
  College: Aalim Mohammed Salegh College of Engineering
  Password Hash: $2b$12$h5WNlplr9flb4...


## 4. User Authentication and Login

Implement secure login functionality that verifies email and password credentials.

In [4]:
class UserAuthentication:
    """Handle user login and authentication"""
    
    def __init__(self):
        # In a real app, this would be your database
        # For demo, we'll use the profile from registration
        self.user_database = {}
        
    def store_user(self, profile_data):
        """Store user in demo database"""
        self.user_database[profile_data['email']] = profile_data
    
    def authenticate_user(self, email: str, password: str) -> dict:
        """
        Authenticate user with email and password
        
        Args:
            email (str): User's email address
            password (str): User's password
            
        Returns:
            dict: Authentication result with user session data
        """
        try:
            # Check if user exists
            if email not in self.user_database:
                return {
                    'success': False,
                    'message': 'User not found',
                    'error': 'INVALID_EMAIL'
                }
            
            user_profile = self.user_database[email]
            
            # Verify password
            if not PasswordManager.verify_password(password, user_profile['password_hash']):
                return {
                    'success': False,
                    'message': 'Invalid password',
                    'error': 'INVALID_PASSWORD'
                }
            
            # Update last login
            user_profile['last_login'] = datetime.now(timezone.utc)
            
            # Return successful authentication
            return {
                'success': True,
                'message': 'Login successful',
                'user': {
                    'id': user_profile['id'],
                    'email': user_profile['email'],
                    'full_name': user_profile['full_name'],
                    'course': user_profile['course'],
                    'college_name': user_profile['college_name'],
                    'profile_completed': user_profile['profile_completed'],
                    'last_login': user_profile['last_login'].isoformat()
                }
            }
            
        except Exception as e:
            return {
                'success': False,
                'message': 'Authentication failed',
                'error': str(e)
            }

# Demo authentication system
auth_system = UserAuthentication()

# Store the demo user we created earlier
auth_system.store_user(registration_result['profile_data'])

print("🔐 User Authentication Demo:")
print("\n✅ Valid Login Test:")
login_result = auth_system.authenticate_user('hamdeen.a2004@gmail.com', 'SecurePassword123!')
print(f"Success: {login_result['success']}")
print(f"Message: {login_result['message']}")
if login_result['success']:
    user = login_result['user']
    print(f"Welcome back, {user['full_name']}!")
    print(f"Last login: {user['last_login']}")

print("\n❌ Invalid Password Test:")
invalid_login = auth_system.authenticate_user('hamdeen.a2004@gmail.com', 'wrongpassword')
print(f"Success: {invalid_login['success']}")
print(f"Message: {invalid_login['message']}")

print("\n❌ Invalid Email Test:")
invalid_email = auth_system.authenticate_user('nonexistent@email.com', 'password')
print(f"Success: {invalid_email['success']}")
print(f"Message: {invalid_email['message']}")

🔐 User Authentication Demo:

✅ Valid Login Test:
Success: True
Message: Login successful
Welcome back, MOHAMED HAMDEEN!
Last login: 2025-09-21T20:15:54.833324+00:00

❌ Invalid Password Test:
Success: False
Message: Invalid password

❌ Invalid Email Test:
Success: False
Message: User not found


## 5. Portfolio Generation Data Management

Handle portfolio generation requests with the JSON structure you provided.

In [5]:
class PortfolioManager:
    """Manage portfolio generation requests and data"""
    
    def __init__(self):
        self.portfolio_database = {}
    
    def create_portfolio_generation(self, user_id: str, portfolio_data: dict, 
                                  template_id: str = "default") -> dict:
        """
        Create a new portfolio generation request
        
        Args:
            user_id (str): User's unique identifier
            portfolio_data (dict): Generated portfolio JSON data
            template_id (str): Portfolio template identifier
            
        Returns:
            dict: Portfolio generation result
        """
        try:
            generation_id = str(uuid.uuid4())
            
            # Store the complete portfolio generation record
            portfolio_record = {
                'id': generation_id,
                'user_id': user_id,
                'template_id': template_id,
                'portfolio_json': portfolio_data,  # Store the complete JSON
                'status': 'completed',
                'created_at': datetime.now(timezone.utc),
                'updated_at': datetime.now(timezone.utc)
            }
            
            # Store in demo database
            self.portfolio_database[generation_id] = portfolio_record
            
            return {
                'success': True,
                'generation_id': generation_id,
                'message': 'Portfolio generation created successfully',
                'portfolio_data': portfolio_record
            }
            
        except Exception as e:
            return {
                'success': False,
                'error': str(e),
                'message': 'Failed to create portfolio generation'
            }
    
    def get_user_portfolios(self, user_id: str) -> list:
        """Get all portfolio generations for a user"""
        user_portfolios = []
        for portfolio in self.portfolio_database.values():
            if portfolio['user_id'] == user_id:
                user_portfolios.append(portfolio)
        return user_portfolios

# Your provided portfolio JSON structure
portfolio_json_data = {
    "output": {
        "name": "MOHAMED HAMDEEN",
        "contact": {
            "email": "hamdeen.a2004@gmail.com",
            "phone": "9043661031",
            "linkedin": None,
            "github": None
        },
        "summary": None,
        "skills": [
            "Python", "JavaScript", "SQL", "HTML", "CSS", "Pandas", "NumPy",
            "Matplotlib", "Seaborn", "Scikit-learn", "PyTorch", "TensorFlow",
            "ReactJS", "Jupyter Notebook", "Power BI", "Tableau", "Excel",
            "MySQL", "Git", "GitHub", "SonarQube", "CI/CD", "GitHub Actions",
            "Playwright", "Unit Testing", "Automated Test Reports",
            "Analytical Thinking", "Problem-Solving", "Teamwork", "Effective Communication"
        ],
        "experience": [
            {
                "role": "Software Engineering Intern",
                "company": "CIEL HR",
                "dates": "08/2025 – 10/2025",
                "responsibilities": [
                    "Contributed to a Contract Lifecycle Management (CLM) platform with role-based access.",
                    "Automated testing using Python Playwright and set up CI/CD pipelines.",
                    "Improved code quality and security with SonarQube integration."
                ]
            },
            {
                "role": "Full Stack Web Developer Intern",
                "company": "Adovi",
                "dates": "05/2024 – 06/2024",
                "responsibilities": [
                    "Built responsive web applications using HTML, CSS, JavaScript, and ReactJS.",
                    "Developed modular, reusable components to improve maintainability.",
                    "Enhanced usability with interactive UI elements and delivered standards-compliant code."
                ]
            }
        ],
        "education": [
            {
                "degree": "B.Tech Artificial Intelligence and Data Science (Honours)",
                "institution": "Aalim Mohammed Salegh College of Engineering   (Anna University)",
                "dates": "2022 – 2026"
            }
        ],
        "projects": [
            {
                "name": "Image Classification with Custom Dataset",
                "description": "Built a CNN from scratch (ResNet-inspired) to classify images from a custom dataset. Implemented data loading pipeline using folder-based structure with transformations and augmentation. Trained and validated the model with train/test split, achieving strong accuracy. Documented workflow for model training, evaluation, and reproducibility.",
                "technologies": ["Python", "PyTorch", "CNN", "Data Augmentation"]
            },
            {
                "name": "Diabetes Prediction System",
                "description": "Developed an end-to-end ML web application to predict diabetes risk using Logistic Regression. Performed data preprocessing and model training, achieving 85%+ accuracy on test data. Integrated the trained model into a Flask-based web interface for real-time predictions. Deployed locally and documented workflow for reproducibility and future improvements.",
                "technologies": ["Python", "Pandas", "Scikit-learn", "Logistic Regression", "Flask"]
            }
        ]
    }
}

# Demo portfolio generation
portfolio_manager = PortfolioManager()

# Create portfolio generation for our demo user
user_id = registration_result['user_id']
portfolio_result = portfolio_manager.create_portfolio_generation(
    user_id=user_id,
    portfolio_data=portfolio_json_data,
    template_id="professional_template_v1"
)

print("📊 Portfolio Generation Demo:")
print(f"Success: {portfolio_result['success']}")
print(f"Generation ID: {portfolio_result.get('generation_id', 'N/A')}")
print(f"Message: {portfolio_result['message']}")

if portfolio_result['success']:
    portfolio = portfolio_result['portfolio_data']
    print(f"\n📋 Portfolio Record:")
    print(f"  User ID: {portfolio['user_id']}")
    print(f"  Template: {portfolio['template_id']}")
    print(f"  Status: {portfolio['status']}")
    print(f"  Created: {portfolio['created_at']}")
    
    # Show portfolio content summary
    output = portfolio['portfolio_json']['output']
    print(f"\n👤 Portfolio Content:")
    print(f"  Name: {output['name']}")
    print(f"  Email: {output['contact']['email']}")
    print(f"  Skills Count: {len(output['skills'])}")
    print(f"  Experience Count: {len(output['experience'])}")
    print(f"  Projects Count: {len(output['projects'])}")

📊 Portfolio Generation Demo:
Success: True
Generation ID: e1e4ea41-7f9f-46fb-ad37-6b8796adba58
Message: Portfolio generation created successfully

📋 Portfolio Record:
  User ID: 4091885c-a53a-4bb8-8a96-ab3a00049903
  Template: professional_template_v1
  Status: completed
  Created: 2025-09-21 20:15:55.086191+00:00

👤 Portfolio Content:
  Name: MOHAMED HAMDEEN
  Email: hamdeen.a2004@gmail.com
  Skills Count: 30
  Experience Count: 2
  Projects Count: 2


## 6. Database Query Operations

Implement CRUD operations for both user_profiles and portfolio_generations tables.

In [ ]:
# SQL queries for your actual database implementation

# 1. INSERT USER PROFILE (with hashed password)
INSERT_USER_PROFILE = """
INSERT INTO public.user_profiles (
    id, email, password_hash, full_name, mobile_number, student_id,
    course, year_of_graduation, college_name, linkedin_profile,
    github_profile, profile_completed, account_status, created_at, updated_at
) VALUES (
    %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s
)
"""

# 2. AUTHENTICATE USER (get user by email for password verification)
GET_USER_BY_EMAIL = """
SELECT id, email, password_hash, full_name, course, college_name, 
       profile_completed, account_status, created_at, last_login
FROM public.user_profiles 
WHERE email = %s AND account_status = 'active'
"""

# 3. UPDATE LAST LOGIN
UPDATE_LAST_LOGIN = """
UPDATE public.user_profiles 
SET last_login = %s, updated_at = %s 
WHERE id = %s
"""

# 4. INSERT PORTFOLIO GENERATION
INSERT_PORTFOLIO_GENERATION = """
INSERT INTO public.portfolio_generations (
    id, user_id, template_id, portfolio_json, status, 
    created_at, updated_at
) VALUES (
    %s, %s, %s, %s, %s, %s, %s
)
"""

# 5. GET USER PORTFOLIOS
GET_USER_PORTFOLIOS = """
SELECT id, template_id, portfolio_json, status, created_at, updated_at
FROM public.portfolio_generations 
WHERE user_id = %s 
ORDER BY created_at DESC
"""

# 6. GET ALL USERS (for admin view)
GET_ALL_USERS = """
SELECT 
    up.id,
    up.email,
    up.full_name,
    up.mobile_number,
    up.course,
    up.year_of_graduation,
    up.college_name,
    up.profile_completed,
    up.account_status,
    up.created_at,
    up.last_login,
    COUNT(pg.id) as portfolio_count
FROM public.user_profiles up
LEFT JOIN public.portfolio_generations pg ON up.id = pg.user_id
GROUP BY up.id, up.email, up.full_name, up.mobile_number, up.course, 
         up.year_of_graduation, up.college_name, up.profile_completed, 
         up.account_status, up.created_at, up.last_login
ORDER BY up.created_at DESC
"""

print("📝 SQL Queries for Database Operations:")
print("✅ User registration with hashed passwords")
print("✅ User authentication and login")
print("✅ Portfolio generation storage")
print("✅ User portfolio retrieval")
print("✅ Admin user management")
print("✅ Last login tracking")

# Demo database operations class
class DatabaseOperations:
    """Production-ready database operations"""
    
    def __init__(self, connection_config):
        self.config = connection_config
        
    def execute_query(self, query, params=None, fetch_one=False, fetch_all=False):
        """Execute database query (demo implementation)"""
        # In production, this would use actual database connection
        print(f"📊 Executing Query: {query[:50]}...")
        if params:
            print(f"🔧 Parameters: {params}")
        
        # Return mock result for demo
        if fetch_one:
            return {"success": True, "data": {"id": "mock_id"}}
        elif fetch_all:
            return {"success": True, "data": [{"id": "mock_id_1"}, {"id": "mock_id_2"}]}
        else:
            return {"success": True, "message": "Query executed successfully"}

# Demo the database operations
db_ops = DatabaseOperations(DB_CONFIG)

print(f"\n🔧 Database Operations Demo:")
print("1. User Registration Query:")
result1 = db_ops.execute_query(INSERT_USER_PROFILE, 
    ['uuid', 'email', 'hash', 'name', 'phone', 'id', 'course', 2026, 'college', 'linkedin', 'github', True, 'active', 'now', 'now'])

print("2. User Authentication Query:")
result2 = db_ops.execute_query(GET_USER_BY_EMAIL, ['hamdeen.a2004@gmail.com'], fetch_one=True)

print("3. Portfolio Generation Query:")
result3 = db_ops.execute_query(INSERT_PORTFOLIO_GENERATION, 
    ['gen_uuid', 'user_uuid', 'template', 'json_data', 'completed', 'now', 'now'])

## 7. Data Validation and Error Handling

Implement comprehensive validation and error handling for all database operations.

In [ ]:
import re
from typing import Optional, Dict, Any

class DataValidator:
    """Comprehensive data validation for user registration and portfolio generation"""
    
    @staticmethod
    def validate_email(email: str) -> Dict[str, Any]:
        """Validate email format"""
        pattern = r'^[a-zA-Z0-9._%+-]+@[a-zA-Z0-9.-]+\.[a-zA-Z]{2,}$'
        if not email or not re.match(pattern, email):
            return {"valid": False, "error": "Invalid email format"}
        return {"valid": True}
    
    @staticmethod
    def validate_password(password: str) -> Dict[str, Any]:
        """Validate password strength"""
        if not password:
            return {"valid": False, "error": "Password is required"}
        if len(password) < 8:
            return {"valid": False, "error": "Password must be at least 8 characters"}
        if not re.search(r'[A-Z]', password):
            return {"valid": False, "error": "Password must contain uppercase letter"}
        if not re.search(r'[a-z]', password):
            return {"valid": False, "error": "Password must contain lowercase letter"}
        if not re.search(r'\d', password):
            return {"valid": False, "error": "Password must contain number"}
        return {"valid": True}
    
    @staticmethod
    def validate_phone(phone: str) -> Dict[str, Any]:
        """Validate phone number"""
        if not phone:
            return {"valid": True}  # Phone is optional
        # Remove all non-digits
        digits_only = re.sub(r'\D', '', phone)
        if len(digits_only) < 10:
            return {"valid": False, "error": "Phone number must be at least 10 digits"}
        return {"valid": True}
    
    @staticmethod
    def validate_year(year: Optional[int]) -> Dict[str, Any]:
        """Validate graduation year"""
        if year is None:
            return {"valid": True}  # Year is optional
        current_year = datetime.now().year
        if year < current_year or year > current_year + 10:
            return {"valid": False, "error": f"Year must be between {current_year} and {current_year + 10}"}
        return {"valid": True}
    
    @classmethod
    def validate_user_registration(cls, user_data: Dict[str, Any]) -> Dict[str, Any]:
        """Validate complete user registration data"""
        errors = []
        
        # Required fields
        required_fields = ['email', 'password', 'full_name', 'course', 'college_name']
        for field in required_fields:
            if not user_data.get(field):
                errors.append(f"{field.replace('_', ' ').title()} is required")
        
        # Email validation
        email_result = cls.validate_email(user_data.get('email', ''))
        if not email_result['valid']:
            errors.append(email_result['error'])
        
        # Password validation
        password_result = cls.validate_password(user_data.get('password', ''))
        if not password_result['valid']:
            errors.append(password_result['error'])
        
        # Phone validation
        phone_result = cls.validate_phone(user_data.get('mobile_number', ''))
        if not phone_result['valid']:
            errors.append(phone_result['error'])
        
        # Year validation
        year_result = cls.validate_year(user_data.get('year_of_graduation'))
        if not year_result['valid']:
            errors.append(year_result['error'])
        
        return {
            'valid': len(errors) == 0,
            'errors': errors
        }

class ErrorHandler:
    """Handle database and application errors"""
    
    @staticmethod
    def handle_database_error(error: Exception) -> Dict[str, Any]:
        """Handle database-specific errors"""
        error_str = str(error).lower()
        
        if 'unique constraint' in error_str or 'duplicate key' in error_str:
            if 'email' in error_str:
                return {
                    'error_type': 'DUPLICATE_EMAIL',
                    'message': 'An account with this email already exists',
                    'user_friendly': True
                }
        
        if 'foreign key constraint' in error_str:
            return {
                'error_type': 'REFERENCE_ERROR',
                'message': 'Invalid reference to user account',
                'user_friendly': False
            }
        
        if 'connection' in error_str:
            return {
                'error_type': 'CONNECTION_ERROR',
                'message': 'Unable to connect to database',
                'user_friendly': True
            }
        
        # Generic database error
        return {
            'error_type': 'DATABASE_ERROR',
            'message': 'A database error occurred. Please try again.',
            'user_friendly': True
        }

# Demo validation and error handling
print("🔍 Data Validation Demo:")

# Test valid data
valid_data = {
    'email': 'hamdeen.a2004@gmail.com',
    'password': 'SecurePass123!',
    'full_name': 'Mohamed Hamdeen',
    'mobile_number': '9043661031',
    'course': 'AI and Data Science',
    'year_of_graduation': 2026,
    'college_name': 'Aalim Mohammed Salegh College'
}

validation_result = DataValidator.validate_user_registration(valid_data)
print(f"✅ Valid Data Test: {validation_result['valid']}")

# Test invalid data
invalid_data = {
    'email': 'invalid-email',
    'password': '123',  # Too short
    'full_name': '',    # Empty
    'mobile_number': '123',  # Too short
    'year_of_graduation': 2020  # Past year
}

invalid_result = DataValidator.validate_user_registration(invalid_data)
print(f"❌ Invalid Data Test: {invalid_result['valid']}")
print("Errors found:")
for error in invalid_result['errors']:
    print(f"  - {error}")

# Demo error handling
print(f"\n🚨 Error Handling Demo:")
mock_db_error = Exception("duplicate key value violates unique constraint 'user_profiles_email_key'")
error_result = ErrorHandler.handle_database_error(mock_db_error)
print(f"Error Type: {error_result['error_type']}")
print(f"User Message: {error_result['message']}")
print(f"User Friendly: {error_result['user_friendly']}")

## Summary

This notebook demonstrates a complete database architecture for your AI Portfolio Generator with:

### 🔐 **Security Features:**
- **Bcrypt password hashing** - Never store plain text passwords
- **Secure authentication** - Proper credential verification
- **Input validation** - Prevent invalid data entry
- **Error handling** - User-friendly error messages

### 📊 **Database Structure:**
- **`user_profiles`** - Complete signup data with hashed passwords
- **`portfolio_generations`** - JSON portfolio data storage
- **Professional organization** - Proper relationships and constraints

### 🎯 **Login Flow:**
1. User enters email and password
2. System retrieves user by email from `user_profiles`
3. Password verified using bcrypt against stored hash
4. Successful login redirects to resume upload page
5. User session contains profile data for personalization

### 🚀 **Next Steps:**
1. **Deploy the updated database schema** using the step-by-step SQL files
2. **Update your AuthService** to use password hashing
3. **Test the complete signup → login → portfolio flow**
4. **View all user data** easily in your Supabase dashboard

Your database will now store all signup form data professionally with secure password handling!